In [1]:
import pandas as pd
import os
import gc
import datetime
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import time
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE,BorderlineSMOTE
from imblearn.combine import SMOTETomek
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [2]:
#企业的基本信息（只包含发债企业）
#2018-2020年的财务指标数据
#2018-2020年的舆情信息（只包含发债企业）
#2019-2020年的违约记录
info_data=pd.read_csv("C:/工作/国泰君安/国泰君安-企业发债风险预测/ent_info.csv",sep="|")
financial_data=pd.read_csv("C:/工作/国泰君安/国泰君安-企业发债风险预测/ent_financial_indicator.csv",sep="|")
res_fin_data=pd.read_csv("C:/工作/国泰君安/国泰君安-企业发债风险预测/res_fin_data.csv",sep="|")
default_data=pd.read_csv("C:/工作/国泰君安/国泰君安-企业发债风险预测/ent_default.csv",sep="|")
news_data=pd.read_csv("C:/工作/国泰君安/国泰君安-企业发债风险预测/ent_news.csv",sep="|",error_bad_lines=False)
answer_data=pd.read_csv("C:/工作/国泰君安/国泰君安-企业发债风险预测/sample_submission/answer.csv",sep="|")
fake_data = pd.read_csv("C:/工作/国泰君安/国泰君安-企业发债风险预测/fake_label.csv",sep="|")
corr_data = pd.read_csv("C:/工作/国泰君安/国泰君安-企业发债风险预测/corr_data.csv",sep="|")

b'Skipping line 108: expected 9 fields, saw 10\nSkipping line 384: expected 9 fields, saw 10\nSkipping line 394: expected 9 fields, saw 11\nSkipping line 496: expected 9 fields, saw 10\nSkipping line 870: expected 9 fields, saw 10\nSkipping line 904: expected 9 fields, saw 10\nSkipping line 1103: expected 9 fields, saw 10\nSkipping line 1165: expected 9 fields, saw 10\nSkipping line 1481: expected 9 fields, saw 10\nSkipping line 1502: expected 9 fields, saw 10\nSkipping line 1648: expected 9 fields, saw 10\nSkipping line 1683: expected 9 fields, saw 10\nSkipping line 1926: expected 9 fields, saw 10\nSkipping line 1936: expected 9 fields, saw 10\nSkipping line 1977: expected 9 fields, saw 10\nSkipping line 1982: expected 9 fields, saw 10\nSkipping line 2304: expected 9 fields, saw 10\nSkipping line 2978: expected 9 fields, saw 10\nSkipping line 3216: expected 9 fields, saw 10\nSkipping line 3319: expected 9 fields, saw 10\nSkipping line 3322: expected 9 fields, saw 10\nSkipping line 366

b'Skipping line 66052: expected 9 fields, saw 10\nSkipping line 66087: expected 9 fields, saw 10\nSkipping line 66114: expected 9 fields, saw 10\nSkipping line 66150: expected 9 fields, saw 10\nSkipping line 66196: expected 9 fields, saw 10\nSkipping line 66247: expected 9 fields, saw 10\nSkipping line 66267: expected 9 fields, saw 10\nSkipping line 66669: expected 9 fields, saw 10\nSkipping line 67028: expected 9 fields, saw 10\nSkipping line 67298: expected 9 fields, saw 10\nSkipping line 67519: expected 9 fields, saw 10\nSkipping line 67571: expected 9 fields, saw 10\nSkipping line 67628: expected 9 fields, saw 10\nSkipping line 67661: expected 9 fields, saw 10\nSkipping line 67969: expected 9 fields, saw 10\nSkipping line 68099: expected 9 fields, saw 10\nSkipping line 68367: expected 9 fields, saw 10\nSkipping line 68415: expected 9 fields, saw 10\nSkipping line 68479: expected 9 fields, saw 10\nSkipping line 68492: expected 9 fields, saw 10\nSkipping line 68548: expected 9 fields

b'Skipping line 131976: expected 9 fields, saw 10\nSkipping line 132119: expected 9 fields, saw 10\nSkipping line 132199: expected 9 fields, saw 10\nSkipping line 132810: expected 9 fields, saw 10\nSkipping line 132950: expected 9 fields, saw 10\nSkipping line 133254: expected 9 fields, saw 10\nSkipping line 133284: expected 9 fields, saw 10\nSkipping line 133373: expected 9 fields, saw 10\nSkipping line 133393: expected 9 fields, saw 10\nSkipping line 133536: expected 9 fields, saw 10\nSkipping line 133663: expected 9 fields, saw 10\nSkipping line 133780: expected 9 fields, saw 10\nSkipping line 133804: expected 9 fields, saw 10\nSkipping line 133893: expected 9 fields, saw 10\nSkipping line 134169: expected 9 fields, saw 10\nSkipping line 134198: expected 9 fields, saw 10\nSkipping line 134595: expected 9 fields, saw 10\nSkipping line 134618: expected 9 fields, saw 10\nSkipping line 134676: expected 9 fields, saw 10\nSkipping line 135142: expected 9 fields, saw 10\nSkipping line 1352

b'Skipping line 198002: expected 9 fields, saw 10\nSkipping line 198061: expected 9 fields, saw 10\nSkipping line 198142: expected 9 fields, saw 10\nSkipping line 198241: expected 9 fields, saw 10\nSkipping line 198586: expected 9 fields, saw 10\nSkipping line 198656: expected 9 fields, saw 10\nSkipping line 198794: expected 9 fields, saw 10\nSkipping line 198899: expected 9 fields, saw 10\nSkipping line 199023: expected 9 fields, saw 10\nSkipping line 199058: expected 9 fields, saw 10\nSkipping line 199101: expected 9 fields, saw 10\nSkipping line 199213: expected 9 fields, saw 10\nSkipping line 199284: expected 9 fields, saw 10\nSkipping line 199392: expected 9 fields, saw 10\nSkipping line 199608: expected 9 fields, saw 10\nSkipping line 199637: expected 9 fields, saw 10\nSkipping line 199796: expected 9 fields, saw 10\nSkipping line 199837: expected 9 fields, saw 10\nSkipping line 199870: expected 9 fields, saw 10\nSkipping line 199878: expected 9 fields, saw 10\nSkipping line 1998

b'Skipping line 264063: expected 9 fields, saw 10\nSkipping line 264152: expected 9 fields, saw 10\nSkipping line 264398: expected 9 fields, saw 10\nSkipping line 264490: expected 9 fields, saw 10\nSkipping line 264672: expected 9 fields, saw 10\nSkipping line 264680: expected 9 fields, saw 10\nSkipping line 264685: expected 9 fields, saw 10\nSkipping line 264702: expected 9 fields, saw 10\nSkipping line 264793: expected 9 fields, saw 10\nSkipping line 264860: expected 9 fields, saw 10\nSkipping line 265324: expected 9 fields, saw 10\nSkipping line 265398: expected 9 fields, saw 10\nSkipping line 265409: expected 9 fields, saw 10\nSkipping line 266023: expected 9 fields, saw 10\nSkipping line 266467: expected 9 fields, saw 10\nSkipping line 266509: expected 9 fields, saw 10\nSkipping line 266666: expected 9 fields, saw 10\nSkipping line 267204: expected 9 fields, saw 10\nSkipping line 267568: expected 9 fields, saw 10\nSkipping line 267649: expected 9 fields, saw 10\nSkipping line 2677

b'Skipping line 329842: expected 9 fields, saw 10\nSkipping line 329898: expected 9 fields, saw 10\nSkipping line 330014: expected 9 fields, saw 10\nSkipping line 330155: expected 9 fields, saw 10\nSkipping line 330282: expected 9 fields, saw 10\nSkipping line 330349: expected 9 fields, saw 10\nSkipping line 330504: expected 9 fields, saw 10\nSkipping line 330749: expected 9 fields, saw 10\nSkipping line 331011: expected 9 fields, saw 10\nSkipping line 331022: expected 9 fields, saw 10\nSkipping line 331061: expected 9 fields, saw 10\nSkipping line 331082: expected 9 fields, saw 10\nSkipping line 331528: expected 9 fields, saw 10\nSkipping line 331593: expected 9 fields, saw 10\nSkipping line 331832: expected 9 fields, saw 10\nSkipping line 331914: expected 9 fields, saw 10\nSkipping line 332007: expected 9 fields, saw 10\nSkipping line 332214: expected 9 fields, saw 11\nSkipping line 332883: expected 9 fields, saw 10\nSkipping line 332897: expected 9 fields, saw 10\nSkipping line 3330

b'Skipping line 395995: expected 9 fields, saw 10\nSkipping line 396019: expected 9 fields, saw 10\nSkipping line 396372: expected 9 fields, saw 10\nSkipping line 396906: expected 9 fields, saw 10\nSkipping line 397314: expected 9 fields, saw 10\nSkipping line 397712: expected 9 fields, saw 10\nSkipping line 397740: expected 9 fields, saw 10\nSkipping line 397752: expected 9 fields, saw 10\nSkipping line 398197: expected 9 fields, saw 10\nSkipping line 398550: expected 9 fields, saw 10\nSkipping line 398584: expected 9 fields, saw 10\nSkipping line 398585: expected 9 fields, saw 10\nSkipping line 399071: expected 9 fields, saw 10\nSkipping line 399175: expected 9 fields, saw 10\nSkipping line 399400: expected 9 fields, saw 10\nSkipping line 400788: expected 9 fields, saw 10\nSkipping line 400822: expected 9 fields, saw 10\nSkipping line 400855: expected 9 fields, saw 10\nSkipping line 400911: expected 9 fields, saw 10\nSkipping line 401090: expected 9 fields, saw 10\nSkipping line 4011

b'Skipping line 461750: expected 9 fields, saw 10\nSkipping line 461829: expected 9 fields, saw 10\nSkipping line 461860: expected 9 fields, saw 10\nSkipping line 462501: expected 9 fields, saw 10\nSkipping line 462581: expected 9 fields, saw 10\nSkipping line 462957: expected 9 fields, saw 11\nSkipping line 463250: expected 9 fields, saw 10\nSkipping line 463705: expected 9 fields, saw 10\nSkipping line 463736: expected 9 fields, saw 10\nSkipping line 463763: expected 9 fields, saw 10\nSkipping line 463867: expected 9 fields, saw 10\nSkipping line 464141: expected 9 fields, saw 10\nSkipping line 464366: expected 9 fields, saw 10\nSkipping line 464969: expected 9 fields, saw 10\nSkipping line 464972: expected 9 fields, saw 10\nSkipping line 465124: expected 9 fields, saw 10\nSkipping line 465513: expected 9 fields, saw 10\nSkipping line 465521: expected 9 fields, saw 10\nSkipping line 465571: expected 9 fields, saw 10\nSkipping line 465822: expected 9 fields, saw 10\nSkipping line 4658

b'Skipping line 527814: expected 9 fields, saw 10\nSkipping line 527862: expected 9 fields, saw 10\nSkipping line 527920: expected 9 fields, saw 10\nSkipping line 528124: expected 9 fields, saw 10\nSkipping line 528333: expected 9 fields, saw 10\nSkipping line 528418: expected 9 fields, saw 10\nSkipping line 528473: expected 9 fields, saw 10\nSkipping line 528746: expected 9 fields, saw 10\nSkipping line 529252: expected 9 fields, saw 10\nSkipping line 529372: expected 9 fields, saw 10\nSkipping line 529387: expected 9 fields, saw 10\nSkipping line 529921: expected 9 fields, saw 10\nSkipping line 530467: expected 9 fields, saw 10\nSkipping line 530903: expected 9 fields, saw 10\nSkipping line 530938: expected 9 fields, saw 10\nSkipping line 531203: expected 9 fields, saw 10\nSkipping line 531561: expected 9 fields, saw 10\nSkipping line 531572: expected 9 fields, saw 10\nSkipping line 531706: expected 9 fields, saw 10\nSkipping line 531994: expected 9 fields, saw 10\nSkipping line 5323

b'Skipping line 593994: expected 9 fields, saw 10\nSkipping line 594017: expected 9 fields, saw 10\nSkipping line 594077: expected 9 fields, saw 10\nSkipping line 594331: expected 9 fields, saw 10\nSkipping line 594406: expected 9 fields, saw 10\nSkipping line 594440: expected 9 fields, saw 11\nSkipping line 595155: expected 9 fields, saw 10\nSkipping line 595478: expected 9 fields, saw 10\nSkipping line 595647: expected 9 fields, saw 10\nSkipping line 595703: expected 9 fields, saw 10\nSkipping line 595746: expected 9 fields, saw 10\nSkipping line 595773: expected 9 fields, saw 10\nSkipping line 595775: expected 9 fields, saw 10\nSkipping line 595835: expected 9 fields, saw 10\nSkipping line 595944: expected 9 fields, saw 10\nSkipping line 595968: expected 9 fields, saw 10\nSkipping line 596005: expected 9 fields, saw 10\nSkipping line 596010: expected 9 fields, saw 10\nSkipping line 596039: expected 9 fields, saw 10\nSkipping line 596478: expected 9 fields, saw 10\nSkipping line 5967

In [3]:
res_fin_data=res_fin_data[["ent_id_year","report_count"]]

In [4]:
# 对类别型变量进行编码
def label_encode(series):
    unique = list(series.unique())
    # unique.sort()
    return series.map(dict(zip(unique, range(series.nunique()))))

In [5]:
#对企业信息数据类别型变量进行处理
ent_info_cat_cols = ['industryphy','industryco','enttype','entstatus','prov','city','county','is_bondissuer'] 
for col in ent_info_cat_cols:
    info_data[col] = label_encode(info_data[col])

In [6]:
#先对企业信息数据中两个存在缺失值的时间特征进行填充
na_col = ["apprdate","opfrom","opto"]
for col in na_col:
    info_data.loc[info_data[col].isnull().values==True,col]=info_data[col].mode().values[0]

In [7]:
#提取企业基本信息里面的日期数据
date_col = list(["esdate","apprdate","opfrom","opto"])
for col in date_col:
    info_data[col+"_"+"year"]=info_data[col].apply(lambda x:str(x)[:4])
    info_data[col+"_"+"year"]=info_data[col+"_"+"year"].apply(lambda x:int(x))
    print(info_data[col+"_"+"year"][:10])

0    2011
1    2017
2    2017
3    1992
4    1995
5    2010
6    2010
7    2005
8    2015
9    2000
Name: esdate_year, dtype: int64
0    2021
1    2021
2    2021
3    2021
4    2020
5    2022
6    2021
7    2021
8    2022
9    2020
Name: apprdate_year, dtype: int64
0    2011
1    2017
2    2017
3    1992
4    2017
5    2010
6    2010
7    2005
8    2015
9    2000
Name: opfrom_year, dtype: int64
0    9999
1    9999
2    9999
3    9999
4    9999
5    9999
6    9999
7    9999
8    9999
9    9999
Name: opto_year, dtype: int64


In [8]:
#提取企业基本信息里面的日期数据
# date_col = list(["esdate","apprdate","opfrom","opto"])
# for col in date_col:
#     info_data[col]=info_data[col].apply(lambda x:str(x)[:4])
#     info_data[col]=info_data[col].apply(lambda x:str(x))
#     info_data[col]=info_data[col].apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d'))
#     print(info_data[col][:10])
#     print(info_data[col].dtype)
#     info_data[col+"_"+"year"]=info_data[col].dt.year
#     info_data[col+"_"+"year"]=info_data[col+"_"+"year"].apply(lambda x:int(x))

In [9]:
info_data[["esdate_year","apprdate_year","opfrom_year","opto_year"]].dtypes

esdate_year      int64
apprdate_year    int64
opfrom_year      int64
opto_year        int64
dtype: object

In [10]:
#企业基本信息数据时间方面的特征构造
info_data["esdate_apprdate_gap"]=info_data["esdate_year"]-info_data["apprdate_year"]
info_data["opfrom_opto_gap"]=info_data["opfrom_year"]-info_data["opto_year"]

In [11]:
news_data["publishdate"]=news_data["publishdate"].apply(lambda x:str(x))
news_data["year"]=news_data["publishdate"].apply(lambda x:datetime.datetime.strptime(x, '%Y%m%d')).dt.year
news_data["year"]=news_data["year"].apply(lambda x:int(x))

In [12]:
news_data.shape

(635349, 10)

In [13]:
news_data.drop(["newscode","publishdate","publishtime","newssource","newstitle"],axis=1,inplace=True)

In [14]:
news_data.head(10)

,ent_id,indextype,index,impscore,year
0,qeda55cd146b46ea96da5fd4efa70089f,514,514102,-1,2020
1,qeda55cd146b46ea96da5fd4efa70089f,514,514102,-1,2019
2,qeda55cd146b46ea96da5fd4efa70089f,513,513128,0,2018
3,qeda55cd146b46ea96da5fd4efa70089f,513,513108,-1,2018
4,qeda55cd146b46ea96da5fd4efa70089f,514,514103,-1,2020
5,qeda55cd146b46ea96da5fd4efa70089f,519,519101,-1,2020
6,qeda55cd146b46ea96da5fd4efa70089f,513,513128,-1,2020
7,qeda55cd146b46ea96da5fd4efa70089f,513,513108,0,2018
8,qeda55cd146b46ea96da5fd4efa70089f,513,513108,0,2020
9,qeda55cd146b46ea96da5fd4efa70089f,513,513123,-1,2020


In [15]:
new_encode_col=["indextype","index","impscore"]
for col in new_encode_col:
    news_data[col]=label_encode(news_data[col])

In [16]:
news_data.head(2)

,ent_id,indextype,index,impscore,year
0,qeda55cd146b46ea96da5fd4efa70089f,0,0,0,2020
1,qeda55cd146b46ea96da5fd4efa70089f,0,0,0,2019


In [17]:
temp_news_data=news_data.groupby(["ent_id","year"]).agg({"indextype":["count","mean"],"index":"mean","impscore":"mean"})
temp_news_data.columns=[col[0]+"_"+col[1] for col in temp_news_data.columns]
temp_news_data.reset_index(inplace=True)
temp_news_data.head(2)

,ent_id,year,indextype_count,indextype_mean,index_mean,impscore_mean
0,pz02551f7157daa47863bf24165b32e8af,2018,1,10.0,50.00,0.0
1,pz02551f7157daa47863bf24165b32e8af,2019,4,1.5,4.75,0.5


In [18]:
financial_data["year"] = financial_data["report_period"].apply(lambda x:str(x)[:4])
financial_data["ent_id_year"] = financial_data["ent_id"]+"_"+financial_data["year"]
financial_data["year"]=financial_data["year"].apply(lambda x:int(x))

In [19]:
#对财务数据进行聚合统计
# def fin_agg(data):
#     fin_agg_col=list(data.columns)
#     fin_agg_col.remove("ent_id")
#     fin_agg_col.remove("report_period")
#     fin_agg_col.remove("year")
#     fin_agg_col.remove("ent_id_year")
#     for c in fin_agg_col:
#         temp=data.groupby("ent_id_year").agg({c:["mean"]})
#         temp.columns=[i[0]+"_"+i[1] for i in temp.columns]
#         temp.reset_index(inplace=True)
#         data=pd.merge(data,temp,on="ent_id_year",how="left")
#     #删除财务原始数据列，只保留聚合统计列
# #     drop_col = [i for i in data.columns if ("_mean" not in i) and 
# #                 ("_std" not in i) and ("_max" not in i) and ("_min" not in i)]
#     drop_col = [i for i in data.columns if ("_mean" not in i)]
#     drop_col.remove("ent_id")
#     drop_col.remove("report_period")
#     drop_col.remove("year")
#     drop_col.remove("ent_id_year")
#     data.drop(drop_col,axis=1,inplace=True)
#     data = data.sort_values("report_period").drop_duplicates(subset=["ent_id_year"],keep="last")
#     return data

In [20]:
# financial_data=fin_agg(financial_data)

In [21]:
financial_data=financial_data.sort_values('report_period').drop_duplicates(subset=['ent_id_year'],
                                                                           keep='last')

In [22]:
#整理2018-2019年违约数据
default_data["label"]=1
default_data["year"]=default_data["acu_date"].apply(lambda x:int(str(x)[:4])-1)
default_data["year"]=default_data["year"].apply(lambda x :str(x))
default_data["ent_id_year"]=default_data["ent_id"]+"_"+default_data["year"]
default_data["year"]=default_data["year"].apply(lambda x :int(x))
default_data=default_data.drop(["acu_date"], axis=1, inplace=False)
default_data

,ent_id,label,year,ent_id_year
0,q00350f4fa8694d82639aec1653cf0306,1,2018,q00350f4fa8694d82639aec1653cf0306_2018
1,q00350f4fa8694d82639aec1653cf0306,1,2018,q00350f4fa8694d82639aec1653cf0306_2018
2,q00350f4fa8694d82639aec1653cf0306,1,2018,q00350f4fa8694d82639aec1653cf0306_2018
3,q00350f4fa8694d82639aec1653cf0306,1,2018,q00350f4fa8694d82639aec1653cf0306_2018
4,q005d1b175fed3b08e45aa7a402ac0072,1,2018,q005d1b175fed3b08e45aa7a402ac0072_2018
5,q005d1b175fed3b08e45aa7a402ac0072,1,2018,q005d1b175fed3b08e45aa7a402ac0072_2018
6,q01e01a62678f34dacfaa178d8866b7b0,1,2018,q01e01a62678f34dacfaa178d8866b7b0_2018
7,q01e01a62678f34dacfaa178d8866b7b0,1,2018,q01e01a62678f34dacfaa178d8866b7b0_2018
8,q01e01a62678f34dacfaa178d8866b7b0,1,2018,q01e01a62678f34dacfaa178d8866b7b0_2018
9,q01e01a62678f34dacfaa178d8866b7b0,1,2018,q01e01a62678f34dacfaa178d8866b7b0_2018


In [23]:
#构造测试集
answer_data["ent_id_year"]=answer_data["ent_id"]+"_"+"2020"
answer_data["label"]=0
answer_data["year"]=2020

In [24]:
answer_data.drop(["default_score"],axis=1,inplace=True)

In [25]:
data=pd.concat([default_data,answer_data],axis=0,ignore_index=True)

In [26]:
answer_2018_id=[i for i in answer_data["ent_id"].unique() if i not in default_data.loc[default_data["year"]==2018,"ent_id"].unique().tolist()]
answer_2018=pd.DataFrame(columns=data.columns)
answer_2018["ent_id"]=answer_2018_id
answer_2018["year"]="2018"
answer_2018["label"]=0
answer_2018["ent_id_year"]=answer_2018["ent_id"]+"_"+answer_2018["year"]
answer_2018["year"]=answer_2018["year"].apply(lambda x:int(x))
data=pd.concat([data,answer_2018],axis=0,ignore_index=True)

answer_2019_id=[i for i in answer_data["ent_id"].unique() if i not in default_data.loc[default_data["year"]==2019,"ent_id"].unique().tolist()]
answer_2019=pd.DataFrame(columns=data.columns)
answer_2019["ent_id"]=answer_2019_id
answer_2019["year"]="2019"
answer_2019["label"]=0
answer_2019["ent_id_year"]=answer_2019["ent_id"]+"_"+answer_2019["year"]
answer_2019["year"]=answer_2019["year"].apply(lambda x:int(x))
data=pd.concat([data,answer_2019],axis=0,ignore_index=True)

In [27]:
#合并2018-2020财务数据和企业基本信息数据
data=pd.merge(data,info_data,on="ent_id",how="left")
data=pd.merge(data,res_fin_data,on="ent_id_year",how="left")
data=pd.merge(data,temp_news_data,on=["ent_id","year"],how="left")
data=pd.merge(data,financial_data,on=["ent_id","year","ent_id_year"],how="left")

In [28]:
#企业基本信息时间方向特征构造
data["esdate_year_gap"]=data["esdate_year"]-data["year"]
data["apprdate_year_gap"]=data["apprdate_year"]-data["year"]
data["opfrom_year_gap"]=data["opfrom_year"]-data["year"]
data["opto_year_gap"]=data["opto_year"]-data["year"]

In [29]:
data.drop_duplicates(inplace=True)

In [30]:
data.drop(["ent_id","opfrom","opto","esdate","apprdate",
          "esdate_year","apprdate_year","opfrom_year","opto_year"],axis=1,inplace=True)

In [31]:
data.shape

(26926, 186)

In [32]:
#特征选择，去除高共线性的特征
def drop_high_linear(df,threshold):
    corr_matrix = df.corr().abs()
    corr_matrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))
    drop_col = []
    for c in corr_matrix.columns:
        if any(corr_matrix[c]>threshold):
            drop_col.append(c)
    return drop_col

In [33]:
drop_col = drop_high_linear(data.drop(["year","ent_id_year","label"],axis=1),0.7)
data.drop(drop_col,axis=1,inplace=True)

In [34]:
#先填充行业缺失值
# data["industryco"].fillna(data["industryco"].mean(),inplace=True)
# data.reset_index(drop=True,inplace=True)
# na_df = pd.DataFrame(columns=["col","na_count"])
# na_df["col"]=data.isna().sum().index
# na_df["na_count"]=data.isna().sum().values
# na_cols=list(na_df.loc[na_df["na_count"]>0,"col"].values)
# for col in na_cols:
#     for row in range(0,data.shape[0]):
#         if np.isnan(data[col].iloc[row]):
#             data[col].iloc[row] = data.loc[(data["industryco"]==data["industryco"].iloc[row])&
#                                       (data["prov"]==data["prov"].iloc[row]),col].dropna(axis=0).mean()

# data.fillna(data.mean(),inplace=True)

In [35]:
# corr_matrix = data.drop(["year","ent_id_year","label"],axis=1).corr().abs().where(np.triu(np.ones(data.drop(["year","ent_id_year","label"],axis=1).corr().abs().shape),k=1).astype(np.bool))
# plt.subplots(figsize=(30, 30))
# sns.heatmap(corr_matrix, vmax=1, square=True, cmap="Blues")
# plt.show()

In [36]:
data.to_csv("C:/工作/国泰君安/国泰君安-企业发债风险预测/temp.csv",sep="|",index=False)

In [37]:
#删除与label相关系数低的列以后，测试集表现略微下降
# data=data.drop(list(corr_data.loc[corr_data["corr"]<=0.001,"col"].values),axis=1)

In [38]:
y_train=data.loc[(data["year"].values==2018) | (data["year"].values==2019),"label"]
y_train=y_train.reset_index(drop=True)
x_train=data.loc[(data["year"].values==2018) | (data["year"].values==2019),:]
x_train=x_train.reset_index(drop=True)
x_train.drop(["year","ent_id_year","label"],axis=1,inplace=True)
x_test=data.loc[data["year"].values==2020,:]
x_test=x_test.reset_index(drop=True)
result=pd.DataFrame(columns=["ent_id","default_score"])
result["ent_id"]=x_test["ent_id_year"]
x_test.drop(["year","ent_id_year","label"],axis=1,inplace=True)

In [39]:
#从test数据集里面提取伪标签数据集,效果略微下降
# x_test["temp_col"] = x_test["ent_id_year"].apply(lambda x :True if x in list(fake_data["col_default"].values) else False)
# lake_data=x_test.loc[x_test["temp_col"].values == True,:]
# x_test=x_test.drop(["temp_col"],axis=1)
# lake_data=lake_data.drop(["temp_col"],axis=1)
# lake_data["label"]=0

In [40]:
#将伪标签数据拼接到训练数据中
# y_train=pd.concat([y_train,lake_data["label"]],axis=0,ignore_index=True)
# lake_data.drop(["year","ent_id_year","label"],axis=1,inplace=True)
# x_train=pd.concat([x_train,lake_data],axis=0, ignore_index=True)

In [41]:
# help(SMOTE)

In [42]:
#对不平衡样本进行处理，对异常少类样本进行合成
# sm = bsmote = BorderlineSMOTE(sampling_strategy=0.1, random_state=42)
# x_train,y_train=sm.fit_resample(x_train,y_train)

In [43]:
x_train.shape

(17963, 99)

In [44]:
y_train.value_counts()

0    17812
1      151
Name: label, dtype: int64

In [45]:
def cv_model(train_x, train_y, test_x, clf_name):

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2022)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]
        
        if clf_name == "xgb":
            xgb_model = xgb.XGBClassifier(n_estimators = 5000, max_depths = 15, 
                                          learning_rate = 0.1, gamma = 0.5, min_child_weight = 5,
                                 subsample = 0.8, colsample_bytree = 0.8, reg_alpha =10, reg_lambda = 10)
            xgb_model.fit(trn_x, trn_y, eval_set = [(val_x, val_y)], eval_metric = "auc", 
                          early_stopping_rounds = 100)
            val_pred = xgb_model.predict_proba(val_x)[:,1]
            
            test_pred = xgb_model.predict_proba(test_x)[:,1]
            print("test_pred",test_pred)
            
        if clf_name == "lgb":
            lgb_model = lgb.LGBMClassifier(boosting_type = "gbdt",objective = "binary",
                                           n_estimators = 5000, num_leaves = 13,learning_rate = 0.1,
                             colsample_bytree= 0.8,bagging_fraction= 0.8,
                bagging_freq=4,
                                          reg_alpha =10, reg_lambda =10 ,
                                           min_child_weight=5,random_state = 50)              
            lgb_model.fit(train_x,train_y,eval_set = [(val_x, val_y)], eval_metric = "auc",
                          early_stopping_rounds = 100)

            val_pred = lgb_model.predict_proba(val_x,num_iteration=lgb_model.best_iteration_)[:,1]
            test_pred = lgb_model.predict_proba(test_x,num_iteration=lgb_model.best_iteration_)[:,1]
            print("test_pred",test_pred)
        if clf_name == "cat":
            cat=CatBoostClassifier(n_estimators=2000,learning_rate = 0.1,depth=7,
                                   l2_leaf_reg=10,colsample_bylevel=0.8, eval_metric = "AUC",
                                  early_stopping_rounds = 100)
            cat.fit(train_x,train_y,eval_set = [(val_x, val_y)])
            val_pred = cat.predict_proba(val_x)[:,1]
            test_pred = cat.predict_proba(test_x)[:,1]
            
        train[valid_index]=val_pred
        test += test_pred / kf.n_splits
        cv_scores.append(roc_auc_score(val_y, val_pred))
        
        print(cv_scores)
       
    print("%s_scotrainre_list:" % clf_name, cv_scores)
    print("%s_score_mean:" % clf_name, np.mean(cv_scores))
    print("%s_score_std:" % clf_name, np.std(cv_scores))
    return train,test

In [46]:
lgb_train,lgb_test = cv_model(x_train, y_train, x_test, "lgb")

************************************ 1 ************************************
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[1]	valid_0's auc: 0.935466	valid_0's binary_logloss: 0.0474867
[2]	valid_0's auc: 0.962861	valid_0's binary_logloss: 0.0446234
[3]	valid_0's auc: 0.968109	valid_0's binary_logloss: 0.0427621
[4]	valid_0's auc: 0.972424	valid_0's binary_logloss: 0.0405217
[5]	valid_0's auc: 0.978179	valid_0's binary_logloss: 0.038367
[6]	valid_0's auc: 0.978949	valid_0's binary_logloss: 0.036276
[7]	valid_0's auc: 0.980461	valid_0's binary_logloss: 0.0352525
[8]	valid_0's auc: 0.980185	valid_0's binary_logloss: 0.0338303
[9]	valid_0's auc: 0.980624	valid_0's binary_logloss: 0.0325281
[10]	valid_0's auc: 0.980416	valid_0's binary_logloss: 0.0314407
[11]	valid_0's auc: 0.980099	valid_0's binary_logloss: 0.030

[134]	valid_0's auc: 0.994068	valid_0's binary_logloss: 0.0157002
[135]	valid_0's auc: 0.994068	valid_0's binary_logloss: 0.0156991
[136]	valid_0's auc: 0.994077	valid_0's binary_logloss: 0.0156981
[137]	valid_0's auc: 0.994059	valid_0's binary_logloss: 0.0156735
[138]	valid_0's auc: 0.994032	valid_0's binary_logloss: 0.015656
[139]	valid_0's auc: 0.994014	valid_0's binary_logloss: 0.0156434
[140]	valid_0's auc: 0.993987	valid_0's binary_logloss: 0.015632
[141]	valid_0's auc: 0.993996	valid_0's binary_logloss: 0.0156127
[142]	valid_0's auc: 0.994023	valid_0's binary_logloss: 0.0155976
[143]	valid_0's auc: 0.994032	valid_0's binary_logloss: 0.0155851
[144]	valid_0's auc: 0.994005	valid_0's binary_logloss: 0.0155738
[145]	valid_0's auc: 0.994005	valid_0's binary_logloss: 0.0155738
[146]	valid_0's auc: 0.994141	valid_0's binary_logloss: 0.0155247
[147]	valid_0's auc: 0.994195	valid_0's binary_logloss: 0.0154698
[148]	valid_0's auc: 0.994213	valid_0's binary_logloss: 0.0154183
[149]	valid_

[316]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[317]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[318]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[319]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[320]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[321]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[322]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[323]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[324]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[325]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[326]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[327]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[328]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[329]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[330]	valid_0's auc: 0.995879	valid_0's binary_logloss: 0.0138518
[331]	vali

[78]	valid_0's auc: 0.984863	valid_0's binary_logloss: 0.0202761
[79]	valid_0's auc: 0.985106	valid_0's binary_logloss: 0.0202252
[80]	valid_0's auc: 0.985349	valid_0's binary_logloss: 0.0201582
[81]	valid_0's auc: 0.985415	valid_0's binary_logloss: 0.0201187
[82]	valid_0's auc: 0.985668	valid_0's binary_logloss: 0.0200457
[83]	valid_0's auc: 0.985911	valid_0's binary_logloss: 0.0200025
[84]	valid_0's auc: 0.986135	valid_0's binary_logloss: 0.019932
[85]	valid_0's auc: 0.986379	valid_0's binary_logloss: 0.0198424
[86]	valid_0's auc: 0.986491	valid_0's binary_logloss: 0.0198115
[87]	valid_0's auc: 0.986631	valid_0's binary_logloss: 0.019756
[88]	valid_0's auc: 0.986884	valid_0's binary_logloss: 0.0197163
[89]	valid_0's auc: 0.987015	valid_0's binary_logloss: 0.0196287
[90]	valid_0's auc: 0.987108	valid_0's binary_logloss: 0.0195508
[91]	valid_0's auc: 0.987202	valid_0's binary_logloss: 0.0195056
[92]	valid_0's auc: 0.987323	valid_0's binary_logloss: 0.0194387
[93]	valid_0's auc: 0.98730

[234]	valid_0's auc: 0.992516	valid_0's binary_logloss: 0.0169544
[235]	valid_0's auc: 0.992525	valid_0's binary_logloss: 0.0169503
[236]	valid_0's auc: 0.992525	valid_0's binary_logloss: 0.0169503
[237]	valid_0's auc: 0.992544	valid_0's binary_logloss: 0.0169465
[238]	valid_0's auc: 0.992553	valid_0's binary_logloss: 0.0169431
[239]	valid_0's auc: 0.992562	valid_0's binary_logloss: 0.0169433
[240]	valid_0's auc: 0.992553	valid_0's binary_logloss: 0.0169435
[241]	valid_0's auc: 0.992534	valid_0's binary_logloss: 0.0169436
[242]	valid_0's auc: 0.992544	valid_0's binary_logloss: 0.0169438
[243]	valid_0's auc: 0.992628	valid_0's binary_logloss: 0.016934
[244]	valid_0's auc: 0.992656	valid_0's binary_logloss: 0.0169252
[245]	valid_0's auc: 0.992693	valid_0's binary_logloss: 0.0169166
[246]	valid_0's auc: 0.992703	valid_0's binary_logloss: 0.0169106
[247]	valid_0's auc: 0.992665	valid_0's binary_logloss: 0.016888
[248]	valid_0's auc: 0.992665	valid_0's binary_logloss: 0.0168875
[249]	valid_

...

[77]	valid_0's auc: 0.98003	valid_0's binary_logloss: 0.0216116
[78]	valid_0's auc: 0.979805	valid_0's binary_logloss: 0.0215979
[79]	valid_0's auc: 0.979562	valid_0's binary_logloss: 0.0215631
[80]	valid_0's auc: 0.979337	valid_0's binary_logloss: 0.0215564
[81]	valid_0's auc: 0.979525	valid_0's binary_logloss: 0.0215178
[82]	valid_0's auc: 0.980264	valid_0's binary_logloss: 0.0214517
[83]	valid_0's auc: 0.98032	valid_0's binary_logloss: 0.0214213
[84]	valid_0's auc: 0.980928	valid_0's binary_logloss: 0.0213615
[85]	valid_0's auc: 0.981087	valid_0's binary_logloss: 0.0213033
[86]	valid_0's auc: 0.981031	valid_0's binary_logloss: 0.0212466
[87]	valid_0's auc: 0.981115	valid_0's binary_logloss: 0.0212302
[88]	valid_0's auc: 0.981649	valid_0's binary_logloss: 0.0211584
[89]	valid_0's auc: 0.982295	valid_0's binary_logloss: 0.0210356
[90]	valid_0's auc: 0.982753	valid_0's binary_logloss: 0.0209572
[91]	valid_0's auc: 0.983137	valid_0's binary_logloss: 0.0208658
[92]	valid_0's auc: 0.98363

[248]	valid_0's auc: 0.987339	valid_0's binary_logloss: 0.0183475
[249]	valid_0's auc: 0.987339	valid_0's binary_logloss: 0.0183466
[250]	valid_0's auc: 0.987339	valid_0's binary_logloss: 0.0183466
[251]	valid_0's auc: 0.987339	valid_0's binary_logloss: 0.0183459
[252]	valid_0's auc: 0.987339	valid_0's binary_logloss: 0.0183459
[253]	valid_0's auc: 0.987292	valid_0's binary_logloss: 0.0183358
[254]	valid_0's auc: 0.987264	valid_0's binary_logloss: 0.0183383
[255]	valid_0's auc: 0.987264	valid_0's binary_logloss: 0.0183314
[256]	valid_0's auc: 0.987264	valid_0's binary_logloss: 0.0183334
[257]	valid_0's auc: 0.987264	valid_0's binary_logloss: 0.0183334
[258]	valid_0's auc: 0.987245	valid_0's binary_logloss: 0.0183125
[259]	valid_0's auc: 0.987226	valid_0's binary_logloss: 0.0183073
[260]	valid_0's auc: 0.987254	valid_0's binary_logloss: 0.0183068
[261]	valid_0's auc: 0.987254	valid_0's binary_logloss: 0.0183068
[262]	valid_0's auc: 0.987254	valid_0's binary_logloss: 0.0183068
[263]	vali

test_pred [0.00352197 0.38717178 0.0005125  ... 0.00052649 0.00121584 0.01263529]
[0.9958794443136332, 0.9930489288053139, 0.9891383665450464, 0.9878906981096762]
************************************ 5 ************************************
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[1]	valid_0's auc: 0.914266	valid_0's binary_logloss: 0.0464543
[2]	valid_0's auc: 0.923587	valid_0's binary_logloss: 0.0440428
[3]	valid_0's auc: 0.950552	valid_0's binary_logloss: 0.0418175
[4]	valid_0's auc: 0.958095	valid_0's binary_logloss: 0.0399048
[5]	valid_0's auc: 0.96509	valid_0's binary_logloss: 0.0380101
[6]	valid_0's auc: 0.966334	valid_0's binary_logloss: 0.0361066
[7]	valid_0's auc: 0.96931	valid_0's binary_logloss: 0.0350415
[8]	valid_0's auc: 0.968973	valid_0's binary_logloss: 0.0338776
[9]	valid_0's auc: 0.96794

[138]	valid_0's auc: 0.987647	valid_0's binary_logloss: 0.0175966
[139]	valid_0's auc: 0.987675	valid_0's binary_logloss: 0.017586
[140]	valid_0's auc: 0.987629	valid_0's binary_logloss: 0.0175763
[141]	valid_0's auc: 0.98761	valid_0's binary_logloss: 0.0175616
[142]	valid_0's auc: 0.987488	valid_0's binary_logloss: 0.0175476
[143]	valid_0's auc: 0.987442	valid_0's binary_logloss: 0.0175309
[144]	valid_0's auc: 0.987395	valid_0's binary_logloss: 0.0175158
[145]	valid_0's auc: 0.987395	valid_0's binary_logloss: 0.0175158
[146]	valid_0's auc: 0.987404	valid_0's binary_logloss: 0.0174674
[147]	valid_0's auc: 0.987423	valid_0's binary_logloss: 0.0174244
[148]	valid_0's auc: 0.987797	valid_0's binary_logloss: 0.0173536
[149]	valid_0's auc: 0.98775	valid_0's binary_logloss: 0.0173132
[150]	valid_0's auc: 0.987666	valid_0's binary_logloss: 0.0173056
[151]	valid_0's auc: 0.987619	valid_0's binary_logloss: 0.0172879
[152]	valid_0's auc: 0.987619	valid_0's binary_logloss: 0.0172876
[153]	valid_0

[307]	valid_0's auc: 0.988724	valid_0's binary_logloss: 0.0162583
[308]	valid_0's auc: 0.988724	valid_0's binary_logloss: 0.0162583
[309]	valid_0's auc: 0.988724	valid_0's binary_logloss: 0.0162583
[310]	valid_0's auc: 0.988724	valid_0's binary_logloss: 0.0162583
[311]	valid_0's auc: 0.988724	valid_0's binary_logloss: 0.0162583
[312]	valid_0's auc: 0.988724	valid_0's binary_logloss: 0.0162583
[313]	valid_0's auc: 0.988724	valid_0's binary_logloss: 0.0162583
[314]	valid_0's auc: 0.988724	valid_0's binary_logloss: 0.0162583
[315]	valid_0's auc: 0.988724	valid_0's binary_logloss: 0.0162583
[316]	valid_0's auc: 0.988724	valid_0's binary_logloss: 0.0162583
[317]	valid_0's auc: 0.988724	valid_0's binary_logloss: 0.0162583
[318]	valid_0's auc: 0.988724	valid_0's binary_logloss: 0.0162583
test_pred [0.0035007  0.39666318 0.00058232 ... 0.00065798 0.00137957 0.01476918]
[0.9958794443136332, 0.9930489288053139, 0.9891383665450464, 0.9878906981096762, 0.9890604529290661]
lgb_scotrainre_list: [0.9

In [47]:
xgb_train,xgb_test = cv_model(x_train, y_train, x_test, "xgb")

************************************ 1 ************************************
[10:45:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_depths" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.57980
[1]	validation_0-auc:0.61061
[2]	validation_0-auc:0.60969
[3]	validation_0-auc:0.65727
[4]	validation_0-auc:0.65684
[5]	validation_0-auc:0.65684
[6]	validation_0-auc:0.67229
[7]	validation_0-auc:0.67242
[8]	validation_0-auc:0.67157
[9]	validation_0-auc:0.73235
[10]	validation_0-auc:0.73249
[11]	validation_0-auc:0.74722
[12]	validation_0-auc:0.74716
[13]	validation_0-auc:0.74655
[14]	validation_0-auc:0.74458
[15]	validation_0-auc:0.75941
[16]	validation_0-auc:0.75933
[17]	

[252]	validation_0-auc:0.97947
[253]	validation_0-auc:0.97947
[254]	validation_0-auc:0.97947
[255]	validation_0-auc:0.97947
[256]	validation_0-auc:0.97962
[257]	validation_0-auc:0.97962
[258]	validation_0-auc:0.97962
[259]	validation_0-auc:0.97980
[260]	validation_0-auc:0.97980
[261]	validation_0-auc:0.97980
[262]	validation_0-auc:0.97980
[263]	validation_0-auc:0.97980
[264]	validation_0-auc:0.97980
[265]	validation_0-auc:0.97980
[266]	validation_0-auc:0.97980
[267]	validation_0-auc:0.97985
[268]	validation_0-auc:0.97969
[269]	validation_0-auc:0.97969
[270]	validation_0-auc:0.97969
[271]	validation_0-auc:0.97969
[272]	validation_0-auc:0.97980
[273]	validation_0-auc:0.97968
[274]	validation_0-auc:0.97968
[275]	validation_0-auc:0.97968
[276]	validation_0-auc:0.97968
[277]	validation_0-auc:0.97968
[278]	validation_0-auc:0.97968
[279]	validation_0-auc:0.97968
[280]	validation_0-auc:0.97968
[281]	validation_0-auc:0.97972
[282]	validation_0-auc:0.97972
[283]	validation_0-auc:0.97972
[284]	va

[517]	validation_0-auc:0.98064
[518]	validation_0-auc:0.98064
[519]	validation_0-auc:0.98064
[520]	validation_0-auc:0.98075
[521]	validation_0-auc:0.98073
[522]	validation_0-auc:0.98073
[523]	validation_0-auc:0.98073
[524]	validation_0-auc:0.98073
[525]	validation_0-auc:0.98073
[526]	validation_0-auc:0.98073
[527]	validation_0-auc:0.98073
[528]	validation_0-auc:0.98073
[529]	validation_0-auc:0.98073
[530]	validation_0-auc:0.98073
[531]	validation_0-auc:0.98073
[532]	validation_0-auc:0.98073
[533]	validation_0-auc:0.98077
[534]	validation_0-auc:0.98077
[535]	validation_0-auc:0.98077
[536]	validation_0-auc:0.98084
[537]	validation_0-auc:0.98087
[538]	validation_0-auc:0.98087
[539]	validation_0-auc:0.98087
[540]	validation_0-auc:0.98087
[541]	validation_0-auc:0.98087
[542]	validation_0-auc:0.98087
[543]	validation_0-auc:0.98087
[544]	validation_0-auc:0.98085
[545]	validation_0-auc:0.98085
[546]	validation_0-auc:0.98085
[547]	validation_0-auc:0.98085
[548]	validation_0-auc:0.98085
[549]	va

[54]	validation_0-auc:0.90353
[55]	validation_0-auc:0.90345
[56]	validation_0-auc:0.90701
[57]	validation_0-auc:0.90714
[58]	validation_0-auc:0.90843
[59]	validation_0-auc:0.91038
[60]	validation_0-auc:0.91009
[61]	validation_0-auc:0.91156
[62]	validation_0-auc:0.91136
[63]	validation_0-auc:0.91223
[64]	validation_0-auc:0.91346
[65]	validation_0-auc:0.91296
[66]	validation_0-auc:0.91294
[67]	validation_0-auc:0.91206
[68]	validation_0-auc:0.91183
[69]	validation_0-auc:0.91819
[70]	validation_0-auc:0.91811
[71]	validation_0-auc:0.91650
[72]	validation_0-auc:0.91296
[73]	validation_0-auc:0.91374
[74]	validation_0-auc:0.91406
[75]	validation_0-auc:0.91461
[76]	validation_0-auc:0.91546
[77]	validation_0-auc:0.91509
[78]	validation_0-auc:0.91516
[79]	validation_0-auc:0.92142
[80]	validation_0-auc:0.92248
[81]	validation_0-auc:0.92271
[82]	validation_0-auc:0.92241
[83]	validation_0-auc:0.92326
[84]	validation_0-auc:0.92343
[85]	validation_0-auc:0.92300
[86]	validation_0-auc:0.92264
[87]	valid

[320]	validation_0-auc:0.94529
[321]	validation_0-auc:0.94529
[322]	validation_0-auc:0.94529
[323]	validation_0-auc:0.94558
[324]	validation_0-auc:0.94558
[325]	validation_0-auc:0.94574
[326]	validation_0-auc:0.94574
[327]	validation_0-auc:0.94574
[328]	validation_0-auc:0.94574
[329]	validation_0-auc:0.94574
[330]	validation_0-auc:0.94574
[331]	validation_0-auc:0.94574
[332]	validation_0-auc:0.94597
[333]	validation_0-auc:0.94597
[334]	validation_0-auc:0.94597
[335]	validation_0-auc:0.94597
[336]	validation_0-auc:0.94597
[337]	validation_0-auc:0.94602
[338]	validation_0-auc:0.94602
[339]	validation_0-auc:0.94602
[340]	validation_0-auc:0.94602
[341]	validation_0-auc:0.94602
[342]	validation_0-auc:0.94602
[343]	validation_0-auc:0.94602
[344]	validation_0-auc:0.94602
[345]	validation_0-auc:0.94602
[346]	validation_0-auc:0.94602
[347]	validation_0-auc:0.94602
[348]	validation_0-auc:0.94602
[349]	validation_0-auc:0.94602
[350]	validation_0-auc:0.94602
[351]	validation_0-auc:0.94602
[352]	va

[585]	validation_0-auc:0.94927
[586]	validation_0-auc:0.94927
[587]	validation_0-auc:0.94927
[588]	validation_0-auc:0.94927
[589]	validation_0-auc:0.94927
[590]	validation_0-auc:0.94927
[591]	validation_0-auc:0.94927
[592]	validation_0-auc:0.94927
[593]	validation_0-auc:0.94927
[594]	validation_0-auc:0.94914
[595]	validation_0-auc:0.94934
[596]	validation_0-auc:0.94934
[597]	validation_0-auc:0.94934
[598]	validation_0-auc:0.94934
[599]	validation_0-auc:0.94934
[600]	validation_0-auc:0.94939
[601]	validation_0-auc:0.94939
[602]	validation_0-auc:0.94939
[603]	validation_0-auc:0.94939
[604]	validation_0-auc:0.94932
[605]	validation_0-auc:0.94932
[606]	validation_0-auc:0.94932
[607]	validation_0-auc:0.94932
[608]	validation_0-auc:0.94932
[609]	validation_0-auc:0.94932
[610]	validation_0-auc:0.94932
[611]	validation_0-auc:0.94932
[612]	validation_0-auc:0.94932
[613]	validation_0-auc:0.94932
[614]	validation_0-auc:0.94932
[615]	validation_0-auc:0.94932
[616]	validation_0-auc:0.94932
[617]	va

[850]	validation_0-auc:0.95116
[851]	validation_0-auc:0.95116
[852]	validation_0-auc:0.95116
[853]	validation_0-auc:0.95116
[854]	validation_0-auc:0.95116
[855]	validation_0-auc:0.95116
[856]	validation_0-auc:0.95116
[857]	validation_0-auc:0.95116
[858]	validation_0-auc:0.95116
[859]	validation_0-auc:0.95116
[860]	validation_0-auc:0.95116
[861]	validation_0-auc:0.95116
[862]	validation_0-auc:0.95116
[863]	validation_0-auc:0.95116
[864]	validation_0-auc:0.95116
[865]	validation_0-auc:0.95116
[866]	validation_0-auc:0.95116
[867]	validation_0-auc:0.95116
[868]	validation_0-auc:0.95116
[869]	validation_0-auc:0.95149
[870]	validation_0-auc:0.95149
[871]	validation_0-auc:0.95149
[872]	validation_0-auc:0.95149
[873]	validation_0-auc:0.95149
[874]	validation_0-auc:0.95149
[875]	validation_0-auc:0.95138
[876]	validation_0-auc:0.95138
[877]	validation_0-auc:0.95138
[878]	validation_0-auc:0.95138
[879]	validation_0-auc:0.95138
[880]	validation_0-auc:0.95138
[881]	validation_0-auc:0.95138
[882]	va

[128]	validation_0-auc:0.94790
[129]	validation_0-auc:0.94870
[130]	validation_0-auc:0.94876
[131]	validation_0-auc:0.94912
[132]	validation_0-auc:0.94919
[133]	validation_0-auc:0.94938
[134]	validation_0-auc:0.94938
[135]	validation_0-auc:0.94938
[136]	validation_0-auc:0.94938
[137]	validation_0-auc:0.94966
[138]	validation_0-auc:0.95057
[139]	validation_0-auc:0.95061
[140]	validation_0-auc:0.95061
[141]	validation_0-auc:0.95079
[142]	validation_0-auc:0.95079
[143]	validation_0-auc:0.95079
[144]	validation_0-auc:0.95079
[145]	validation_0-auc:0.95089
[146]	validation_0-auc:0.95087
[147]	validation_0-auc:0.95086
[148]	validation_0-auc:0.95086
[149]	validation_0-auc:0.95107
[150]	validation_0-auc:0.95107
[151]	validation_0-auc:0.95121
[152]	validation_0-auc:0.95121
[153]	validation_0-auc:0.95216
[154]	validation_0-auc:0.95211
[155]	validation_0-auc:0.95191
[156]	validation_0-auc:0.95192
[157]	validation_0-auc:0.95212
[158]	validation_0-auc:0.95250
[159]	validation_0-auc:0.95250
[160]	va

[111]	validation_0-auc:0.95678
[112]	validation_0-auc:0.95661
[113]	validation_0-auc:0.95738
[114]	validation_0-auc:0.95762
[115]	validation_0-auc:0.95753
[116]	validation_0-auc:0.95751
[117]	validation_0-auc:0.95723
[118]	validation_0-auc:0.95729
[119]	validation_0-auc:0.95730
[120]	validation_0-auc:0.95722
[121]	validation_0-auc:0.95674
[122]	validation_0-auc:0.95664
[123]	validation_0-auc:0.95664
[124]	validation_0-auc:0.95721
[125]	validation_0-auc:0.95756
[126]	validation_0-auc:0.95789
[127]	validation_0-auc:0.95833
[128]	validation_0-auc:0.95827
[129]	validation_0-auc:0.95827
[130]	validation_0-auc:0.95814
[131]	validation_0-auc:0.95814
[132]	validation_0-auc:0.95821
[133]	validation_0-auc:0.95901
[134]	validation_0-auc:0.95901
[135]	validation_0-auc:0.95919
[136]	validation_0-auc:0.95919
[137]	validation_0-auc:0.95916
[138]	validation_0-auc:0.95898
[139]	validation_0-auc:0.95894
[140]	validation_0-auc:0.95924
[141]	validation_0-auc:0.95938
[142]	validation_0-auc:0.95911
[143]	va

[376]	validation_0-auc:0.96295
[377]	validation_0-auc:0.96295
[378]	validation_0-auc:0.96285
[379]	validation_0-auc:0.96285
[380]	validation_0-auc:0.96285
[381]	validation_0-auc:0.96285
[382]	validation_0-auc:0.96285
[383]	validation_0-auc:0.96282
[384]	validation_0-auc:0.96282
[385]	validation_0-auc:0.96282
[386]	validation_0-auc:0.96282
[387]	validation_0-auc:0.96282
[388]	validation_0-auc:0.96282
[389]	validation_0-auc:0.96282
[390]	validation_0-auc:0.96282
[391]	validation_0-auc:0.96282
[392]	validation_0-auc:0.96282
[393]	validation_0-auc:0.96282
[394]	validation_0-auc:0.96282
[395]	validation_0-auc:0.96282
[396]	validation_0-auc:0.96282
[397]	validation_0-auc:0.96270
[398]	validation_0-auc:0.96270
test_pred [0.00282332 0.31815135 0.0006791  ... 0.00109918 0.00207457 0.00775371]
[0.9810816685080872, 0.9514921882308914, 0.9528113013378239, 0.9635036496350367]
************************************ 5 ************************************
[10:47:41] WARNING: C:/Users/Administrator/works

In [48]:
cat_train,cat_test = cv_model(x_train, y_train, x_test, "cat")

************************************ 1 ************************************
0:	test: 0.6752866	best: 0.6752866 (0)	total: 225ms	remaining: 7m 28s
1:	test: 0.8253654	best: 0.8253654 (1)	total: 300ms	remaining: 4m 59s
2:	test: 0.9109281	best: 0.9109281 (2)	total: 367ms	remaining: 4m 4s
3:	test: 0.9203465	best: 0.9203465 (3)	total: 442ms	remaining: 3m 40s
4:	test: 0.9220309	best: 0.9220309 (4)	total: 518ms	remaining: 3m 26s
5:	test: 0.9550905	best: 0.9550905 (5)	total: 596ms	remaining: 3m 18s
6:	test: 0.9606555	best: 0.9606555 (6)	total: 669ms	remaining: 3m 10s
7:	test: 0.9729538	best: 0.9729538 (7)	total: 740ms	remaining: 3m 4s
8:	test: 0.9733477	best: 0.9733477 (8)	total: 820ms	remaining: 3m 1s
9:	test: 0.9793112	best: 0.9793112 (9)	total: 892ms	remaining: 2m 57s
10:	test: 0.9792840	best: 0.9793112 (9)	total: 968ms	remaining: 2m 54s
11:	test: 0.9815616	best: 0.9815616 (11)	total: 1.04s	remaining: 2m 52s
12:	test: 0.9814530	best: 0.9815616 (11)	total: 1.11s	remaining: 2m 50s
13:	test: 0.

113:	test: 0.9954900	best: 0.9955263 (108)	total: 8.68s	remaining: 2m 23s
114:	test: 0.9955081	best: 0.9955263 (108)	total: 8.75s	remaining: 2m 23s
115:	test: 0.9954900	best: 0.9955263 (108)	total: 8.83s	remaining: 2m 23s
116:	test: 0.9954810	best: 0.9955263 (108)	total: 8.91s	remaining: 2m 23s
117:	test: 0.9954991	best: 0.9955263 (108)	total: 8.98s	remaining: 2m 23s
118:	test: 0.9955534	best: 0.9955534 (118)	total: 9.06s	remaining: 2m 23s
119:	test: 0.9955715	best: 0.9955715 (119)	total: 9.13s	remaining: 2m 23s
120:	test: 0.9955806	best: 0.9955806 (120)	total: 9.21s	remaining: 2m 23s
121:	test: 0.9955715	best: 0.9955806 (120)	total: 9.29s	remaining: 2m 23s
122:	test: 0.9955806	best: 0.9955806 (120)	total: 9.37s	remaining: 2m 23s
123:	test: 0.9955625	best: 0.9955806 (120)	total: 9.41s	remaining: 2m 22s
124:	test: 0.9955715	best: 0.9955806 (120)	total: 9.48s	remaining: 2m 22s
125:	test: 0.9955715	best: 0.9955806 (120)	total: 9.56s	remaining: 2m 22s
126:	test: 0.9956530	best: 0.9956530 (

226:	test: 0.9974914	best: 0.9975458 (214)	total: 16.9s	remaining: 2m 12s
227:	test: 0.9975548	best: 0.9975548 (227)	total: 17s	remaining: 2m 11s
228:	test: 0.9975458	best: 0.9975548 (227)	total: 17s	remaining: 2m 11s
229:	test: 0.9975458	best: 0.9975548 (227)	total: 17.1s	remaining: 2m 11s
230:	test: 0.9975458	best: 0.9975548 (227)	total: 17.1s	remaining: 2m 11s
231:	test: 0.9975639	best: 0.9975639 (231)	total: 17.2s	remaining: 2m 11s
232:	test: 0.9976001	best: 0.9976001 (232)	total: 17.3s	remaining: 2m 10s
233:	test: 0.9977812	best: 0.9977812 (233)	total: 17.3s	remaining: 2m 10s
234:	test: 0.9977903	best: 0.9977903 (234)	total: 17.4s	remaining: 2m 10s
235:	test: 0.9977903	best: 0.9977903 (234)	total: 17.4s	remaining: 2m 10s
236:	test: 0.9978627	best: 0.9978627 (236)	total: 17.5s	remaining: 2m 10s
237:	test: 0.9978809	best: 0.9978809 (237)	total: 17.6s	remaining: 2m 10s
238:	test: 0.9978899	best: 0.9978899 (238)	total: 17.6s	remaining: 2m 9s
239:	test: 0.9978899	best: 0.9978899 (238)	

339:	test: 0.9990310	best: 0.9990944 (329)	total: 23.9s	remaining: 1m 56s
340:	test: 0.9990310	best: 0.9990944 (329)	total: 24s	remaining: 1m 56s
341:	test: 0.9990582	best: 0.9990944 (329)	total: 24.1s	remaining: 1m 56s
342:	test: 0.9991216	best: 0.9991216 (342)	total: 24.2s	remaining: 1m 56s
343:	test: 0.9991216	best: 0.9991216 (342)	total: 24.3s	remaining: 1m 56s
344:	test: 0.9991216	best: 0.9991216 (342)	total: 24.3s	remaining: 1m 56s
345:	test: 0.9991216	best: 0.9991216 (342)	total: 24.4s	remaining: 1m 56s
346:	test: 0.9991216	best: 0.9991216 (342)	total: 24.5s	remaining: 1m 56s
347:	test: 0.9991306	best: 0.9991306 (347)	total: 24.5s	remaining: 1m 56s
348:	test: 0.9991487	best: 0.9991487 (348)	total: 24.6s	remaining: 1m 56s
349:	test: 0.9991759	best: 0.9991759 (349)	total: 24.7s	remaining: 1m 56s
350:	test: 0.9991759	best: 0.9991759 (349)	total: 24.7s	remaining: 1m 56s
351:	test: 0.9991849	best: 0.9991849 (351)	total: 24.8s	remaining: 1m 56s
352:	test: 0.9991849	best: 0.9991849 (35

453:	test: 0.9995562	best: 0.9996015 (450)	total: 31.2s	remaining: 1m 46s
454:	test: 0.9995744	best: 0.9996015 (450)	total: 31.3s	remaining: 1m 46s
455:	test: 0.9995744	best: 0.9996015 (450)	total: 31.3s	remaining: 1m 46s
456:	test: 0.9995744	best: 0.9996015 (450)	total: 31.4s	remaining: 1m 45s
457:	test: 0.9995834	best: 0.9996015 (450)	total: 31.4s	remaining: 1m 45s
458:	test: 0.9995925	best: 0.9996015 (450)	total: 31.5s	remaining: 1m 45s
459:	test: 0.9995925	best: 0.9996015 (450)	total: 31.6s	remaining: 1m 45s
460:	test: 0.9996015	best: 0.9996015 (450)	total: 31.6s	remaining: 1m 45s
461:	test: 0.9996015	best: 0.9996015 (450)	total: 31.7s	remaining: 1m 45s
462:	test: 0.9996015	best: 0.9996015 (450)	total: 31.8s	remaining: 1m 45s
463:	test: 0.9996015	best: 0.9996015 (450)	total: 31.8s	remaining: 1m 45s
464:	test: 0.9996015	best: 0.9996015 (450)	total: 31.9s	remaining: 1m 45s
465:	test: 0.9996196	best: 0.9996196 (465)	total: 31.9s	remaining: 1m 45s
466:	test: 0.9996196	best: 0.9996196 (

...

152:	test: 0.9892663	best: 0.9892757 (150)	total: 9.27s	remaining: 1m 51s
153:	test: 0.9897529	best: 0.9897529 (153)	total: 9.33s	remaining: 1m 51s
154:	test: 0.9897997	best: 0.9897997 (154)	total: 9.39s	remaining: 1m 51s
155:	test: 0.9900337	best: 0.9900337 (155)	total: 9.45s	remaining: 1m 51s
156:	test: 0.9899869	best: 0.9900337 (155)	total: 9.51s	remaining: 1m 51s
157:	test: 0.9900056	best: 0.9900337 (155)	total: 9.57s	remaining: 1m 51s
158:	test: 0.9900805	best: 0.9900805 (158)	total: 9.63s	remaining: 1m 51s
159:	test: 0.9898185	best: 0.9900805 (158)	total: 9.69s	remaining: 1m 51s
160:	test: 0.9897810	best: 0.9900805 (158)	total: 9.75s	remaining: 1m 51s
161:	test: 0.9898091	best: 0.9900805 (158)	total: 9.81s	remaining: 1m 51s
162:	test: 0.9898278	best: 0.9900805 (158)	total: 9.87s	remaining: 1m 51s
163:	test: 0.9900805	best: 0.9900805 (158)	total: 9.93s	remaining: 1m 51s
164:	test: 0.9902021	best: 0.9902021 (164)	total: 9.99s	remaining: 1m 51s
165:	test: 0.9902489	best: 0.9902489 (

263:	test: 0.9944507	best: 0.9944881 (259)	total: 16s	remaining: 1m 45s
264:	test: 0.9944507	best: 0.9944881 (259)	total: 16.1s	remaining: 1m 45s
265:	test: 0.9944507	best: 0.9944881 (259)	total: 16.2s	remaining: 1m 45s
266:	test: 0.9946659	best: 0.9946659 (266)	total: 16.2s	remaining: 1m 45s
267:	test: 0.9946659	best: 0.9946659 (266)	total: 16.3s	remaining: 1m 45s
268:	test: 0.9946659	best: 0.9946659 (266)	total: 16.3s	remaining: 1m 45s
269:	test: 0.9946659	best: 0.9946659 (266)	total: 16.4s	remaining: 1m 45s
270:	test: 0.9946659	best: 0.9946659 (266)	total: 16.5s	remaining: 1m 44s
271:	test: 0.9946940	best: 0.9946940 (271)	total: 16.5s	remaining: 1m 44s
272:	test: 0.9948624	best: 0.9948624 (272)	total: 16.6s	remaining: 1m 44s
273:	test: 0.9948624	best: 0.9948624 (272)	total: 16.6s	remaining: 1m 44s
274:	test: 0.9952555	best: 0.9952555 (274)	total: 16.7s	remaining: 1m 44s
275:	test: 0.9952555	best: 0.9952555 (274)	total: 16.8s	remaining: 1m 44s
276:	test: 0.9953771	best: 0.9953771 (27

374:	test: 0.9983998	best: 0.9983998 (373)	total: 23.5s	remaining: 1m 42s
375:	test: 0.9983998	best: 0.9983998 (373)	total: 23.6s	remaining: 1m 41s
376:	test: 0.9984091	best: 0.9984091 (376)	total: 23.7s	remaining: 1m 41s
377:	test: 0.9984091	best: 0.9984091 (376)	total: 23.8s	remaining: 1m 42s
378:	test: 0.9984185	best: 0.9984185 (378)	total: 23.8s	remaining: 1m 42s
379:	test: 0.9984185	best: 0.9984185 (378)	total: 23.9s	remaining: 1m 41s
380:	test: 0.9984091	best: 0.9984185 (378)	total: 24s	remaining: 1m 41s
381:	test: 0.9984278	best: 0.9984278 (381)	total: 24.1s	remaining: 1m 41s
382:	test: 0.9984372	best: 0.9984372 (382)	total: 24.1s	remaining: 1m 41s
383:	test: 0.9985214	best: 0.9985214 (383)	total: 24.2s	remaining: 1m 41s
384:	test: 0.9985308	best: 0.9985308 (384)	total: 24.3s	remaining: 1m 41s
385:	test: 0.9985776	best: 0.9985776 (385)	total: 24.4s	remaining: 1m 41s
386:	test: 0.9986431	best: 0.9986431 (386)	total: 24.5s	remaining: 1m 41s
387:	test: 0.9986431	best: 0.9986431 (38

486:	test: 0.9998128	best: 0.9998128 (486)	total: 32.7s	remaining: 1m 41s
487:	test: 0.9998222	best: 0.9998222 (487)	total: 32.7s	remaining: 1m 41s
488:	test: 0.9998409	best: 0.9998409 (488)	total: 32.8s	remaining: 1m 41s
489:	test: 0.9998409	best: 0.9998409 (488)	total: 32.9s	remaining: 1m 41s
490:	test: 0.9998409	best: 0.9998409 (488)	total: 32.9s	remaining: 1m 41s
491:	test: 0.9998503	best: 0.9998503 (491)	total: 33s	remaining: 1m 41s
492:	test: 0.9998503	best: 0.9998503 (491)	total: 33.1s	remaining: 1m 41s
493:	test: 0.9998503	best: 0.9998503 (491)	total: 33.1s	remaining: 1m 40s
494:	test: 0.9998596	best: 0.9998596 (494)	total: 33.2s	remaining: 1m 40s
495:	test: 0.9998596	best: 0.9998596 (494)	total: 33.2s	remaining: 1m 40s
496:	test: 0.9998596	best: 0.9998596 (494)	total: 33.3s	remaining: 1m 40s
497:	test: 0.9999064	best: 0.9999064 (497)	total: 33.3s	remaining: 1m 40s
498:	test: 0.9999158	best: 0.9999158 (498)	total: 33.4s	remaining: 1m 40s
499:	test: 0.9999158	best: 0.9999158 (49

599:	test: 1.0000000	best: 1.0000000 (546)	total: 39.9s	remaining: 1m 33s
600:	test: 1.0000000	best: 1.0000000 (546)	total: 39.9s	remaining: 1m 32s
601:	test: 1.0000000	best: 1.0000000 (546)	total: 40s	remaining: 1m 32s
602:	test: 1.0000000	best: 1.0000000 (546)	total: 40.1s	remaining: 1m 32s
603:	test: 1.0000000	best: 1.0000000 (546)	total: 40.1s	remaining: 1m 32s
604:	test: 1.0000000	best: 1.0000000 (546)	total: 40.2s	remaining: 1m 32s
605:	test: 1.0000000	best: 1.0000000 (546)	total: 40.2s	remaining: 1m 32s
606:	test: 1.0000000	best: 1.0000000 (546)	total: 40.3s	remaining: 1m 32s
607:	test: 1.0000000	best: 1.0000000 (546)	total: 40.4s	remaining: 1m 32s
608:	test: 1.0000000	best: 1.0000000 (546)	total: 40.4s	remaining: 1m 32s
609:	test: 1.0000000	best: 1.0000000 (546)	total: 40.5s	remaining: 1m 32s
610:	test: 1.0000000	best: 1.0000000 (546)	total: 40.6s	remaining: 1m 32s
611:	test: 1.0000000	best: 1.0000000 (546)	total: 40.6s	remaining: 1m 32s
612:	test: 1.0000000	best: 1.0000000 (54

In [49]:
def stack_model(m1_train,m2_train,m3_train,m1_test,m2_test,m3_test,train_y):
    
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2022)
    train_stack=np.vstack([m1_train,m2_train,m3_train]).T
    test_stack=np.vstack([m1_test,m2_test,m3_test]).T
    
    train = np.zeros(train_stack.shape[0])
    test = np.zeros(test_stack.shape[0])

    cv_scores = []
    for i,(train_index,valid_index) in enumerate(kf.split(train_stack,train_y)):
        print(f"运行到第{i}个验证集")
        trn_x, trn_y, val_x, val_y = train_stack[train_index], train_y[train_index], train_stack[valid_index], train_y[valid_index]

        lr=LogisticRegression()
        lr.fit(trn_x,trn_y)
        val_pred = lr.predict_proba(val_x)[:,1]
        test_pred = lr.predict_proba(test_stack)[:,1]
            
        train[valid_index]=val_pred
        test += test_pred / kf.n_splits
        cv_scores.append(roc_auc_score(val_y, val_pred))
        
        print(cv_scores)
        
    print("score_mean:%s" % np.mean(cv_scores))
    print("score_std:%s:" % np.std(cv_scores))
    return test

In [50]:
#stack模型融合
model_stack_res=stack_model(xgb_train,lgb_train,cat_train,xgb_test,lgb_test,cat_test,y_train)

运行到第0个验证集
[0.9999275506692508]
运行到第1个验证集
[0.9999275506692508, 0.9998783796426233]
运行到第2个验证集
[0.9999275506692508, 0.9998783796426233, 0.9999345121152586]
运行到第3个验证集
[0.9999275506692508, 0.9998783796426233, 0.9999345121152586, 0.9998502713831181]
运行到第4个验证集
[0.9999275506692508, 0.9998783796426233, 0.9999345121152586, 0.9998502713831181, 0.999766049036122]
score_mean:0.9998713525692746
score_std:6.12045504090613e-05:


In [51]:
result["default_score"]=model_stack_res

In [52]:
result["ent_id"]=result["ent_id"].apply(lambda x:x[:-5])

In [53]:
result.drop_duplicates(subset=["ent_id"],keep="first",inplace=True)

In [54]:
result.to_csv("C:/工作/国泰君安/国泰君安-企业发债风险预测/answer.csv",sep="|",index=False)